# GWP 3

## Step 2

## Denosing

### Shrinkage Estimator (LW and OAS)

In [202]:
import numpy as np
import pandas as pd
from scipy.linalg import eigh
from sklearn.neighbors import KernelDensity
from sklearn.covariance import LedoitWolf, OAS
from scipy.optimize import minimize
import yfinance as yf
import cvxpy as cp
from sklearn.cluster import AgglomerativeClustering

In [203]:
# Fetch data
tickers = ['AAPL', 'NVDA', 'TSLA', 'XOM', 'REGN', 'LLY', 'JPM']
data = yf.download(tickers, start='2022-01-01', end='2024-12-31',auto_adjust=False)['Adj Close']
returns = data.pct_change().dropna()

[*********************100%***********************]  7 of 7 completed


In [204]:
returns.head()

Ticker          AAPL       JPM       LLY      NVDA      REGN      TSLA  \
Date                                                                     
2022-01-04 -0.012692  0.037910 -0.018106 -0.027589 -0.016393 -0.041833   
2022-01-05 -0.026600 -0.018282 -0.023088 -0.057562 -0.035180 -0.053471   
2022-01-06 -0.016693  0.010624 -0.005064  0.020794  0.005579 -0.021523   
2022-01-07  0.000988  0.009908  0.000656 -0.033040  0.008840 -0.035447   
2022-01-10  0.000116  0.000957  0.002351  0.005615  0.029053  0.030342   

Ticker           XOM  
Date                  
2022-01-04  0.037614  
2022-01-05  0.012437  
2022-01-06  0.023521  
2022-01-07  0.008197  
2022-01-10 -0.005952

In [205]:
# Annualized expected returns
exp_returns = returns.mean() * 252
# Sample covariance matrix
sample_cov = returns.cov().values * 252

In [206]:
# Ledoit-Wolf shrinkage
lw = LedoitWolf()
lw_cov = lw.fit(returns).covariance_ * 252

# OAS shrinkage
oas = OAS()
oas_cov = oas.fit(returns).covariance_ * 252

In [207]:
# Best portfolio from GWP2
best_weights = np.array([-2.8957, -0.4408, 3.1467, 2.3461, -2.4048, -0.2035, 1.4520])

In [208]:
def optimize_portfolio(exp_returns, cov_matrix):
    n = len(exp_returns)
    w = cp.Variable(n)
    risk = cp.quad_form(w, cov_matrix)
    objective = cp.Maximize(exp_returns.values.T @ w - 0.5 * risk)
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return w.value.round(4)

# Optimize with sample covariance
#weights_sample = optimize_portfolio(exp_returns, sample_cov)

# Optimize with Ledoit-Wolf
weights_lw = optimize_portfolio(exp_returns, lw_cov)

# Optimize with OAS
weights_oas = optimize_portfolio(exp_returns, oas_cov)

In [209]:
print(f"Original Weights: {best_weights}")
print(f"Ledoit-Wolf Weights: {weights_lw}")
print(f"OAS Weights: {weights_oas}")

Original Weights: [-2.8957 -0.4408  3.1467  2.3461 -2.4048 -0.2035  1.452 ]
Ledoit-Wolf Weights: [ 0.     -0.      0.0247  0.9753 -0.      0.      0.    ]
OAS Weights: [-0.     -0.      0.0272  0.9728 -0.     -0.      0.    ]


In [210]:
def calculate_metrics(weights, returns_series, cov_matrix, risk_free_rate=0.0453):
    # Portfolio returns (time series)
    portfolio_returns = np.dot(returns_series, weights)

    # Annualized return (geometric mean)
    annualized_return = (1 + portfolio_returns).prod()**(252/len(returns_series)) - 1

    # Annualized volatility
    annualized_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Sharpe ratio (annualized)
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_vol

    # Maximum drawdown
    cumulative_returns = pd.Series((1 + portfolio_returns).cumprod())
    peak = cumulative_returns.expanding().max()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()

    return annualized_return, annualized_vol, sharpe_ratio, max_drawdown

# Calculate metrics for each portfolio
sample_ret, sample_vol, sample_sharpe, sample_dd = calculate_metrics(best_weights, returns.values, sample_cov)
lw_ret, lw_vol, lw_sharpe, lw_dd = calculate_metrics(weights_lw, returns.values, lw_cov)
oas_ret, oas_vol, oas_sharpe, oas_dd = calculate_metrics(weights_oas, returns.values, oas_cov)


print(f"Original Portfolio: Portfolio Return={sample_ret:.2%}, Portfolio Variance={sample_vol:.2%}, Sharpe={sample_sharpe:.2f}, max drawdown={sample_dd:.2%}")
print(f"LW Portfolio: LW Portfolio Return={lw_ret:.2%}, LW Portfolio Variance={lw_vol:.2%}, LW Sharpe={lw_sharpe:.2f}, LW max drawdown={lw_dd:.2%}")
print(f"OAS Portfolio: LW Portfolio Return={oas_ret:.2%}, LW Portfolio Variance={oas_vol:.2%}, LW Sharpe={oas_sharpe:.2f}, OAS max drawdown={oas_dd:.2%}")

Original Portfolio: Portfolio Return=272.40%, Portfolio Variance=150.34%, Sharpe=1.78, max drawdown=-86.46%
LW Portfolio: LW Portfolio Return=66.53%, LW Portfolio Variance=53.83%, LW Sharpe=1.15, LW max drawdown=-60.36%
OAS Portfolio: LW Portfolio Return=66.53%, LW Portfolio Variance=53.80%, LW Sharpe=1.15, OAS max drawdown=-60.23%


### De Prado's Constant Residual Eigenvalue Method (CREM) Denosing

In [211]:
def mpPDF(var, q, pts=1000):
    eMin, eMax = var * (1 - (1./q)**0.5)**2, var * (1 + (1./q)**0.5)**2
    eVal = np.linspace(eMin, eMax, pts)
    pdf = q / (2 * np.pi * var * eVal) * ((eMax - eVal) * (eVal - eMin))**0.5
    return pd.Series(pdf, index=eVal)

def getPCA(matrix):
    eVal, eVec = eigh(matrix)
    indices = eVal.argsort()[::-1]
    eVal, eVec = eVal[indices], eVec[:, indices]
    return np.diag(eVal), eVec

def fitKDE(obs, bWidth=0.01, kernel='gaussian', x=None):
    if len(obs.shape) == 1:
        obs = obs.reshape(-1, 1)
    kde = KernelDensity(kernel=kernel, bandwidth=bWidth).fit(obs)
    if x is None:
        x = np.unique(obs).reshape(-1, 1)
    elif len(x.shape) == 1:
        x = x.reshape(-1, 1)
    logProb = kde.score_samples(x)
    return pd.Series(np.exp(logProb), index=x.flatten())

def findMaxEval(eVal, q, bWidth=0.01):
    def errPDFs(var, eVal, q, bWidth):
        var = var[0]
        pdf0 = mpPDF(var, q, 1000)
        pdf1 = fitKDE(eVal, bWidth, x=pdf0.index.values)
        sse = np.sum((pdf1 - pdf0)**2)
        return sse
    result = minimize(errPDFs, x0=np.array(0.5), args=(eVal, q, bWidth), bounds=[(1e-5, 1-1e-5)])
    var = result.x[0] if result.success else 1.0
    return var * (1 + (1./q)**0.5)**2, var

def denoisedCorr(eVal, eVec, nFacts):
    eVal_ = np.diag(eVal).copy()
    eVal_[nFacts:] = eVal_[nFacts:].sum() / (eVal_.shape[0] - nFacts)
    corr1 = np.dot(eVec, np.diag(eVal_)).dot(eVec.T)
    corr1 = cov2corr(corr1)
    return corr1

def cov2corr(cov):
    std = np.sqrt(np.diag(cov))
    return cov / np.outer(std, std)

In [212]:
# Denoise Covariance Matrix
cov_matrix = returns.cov().values * 252  # Annualized
corr_matrix = cov2corr(cov_matrix)
eVal, eVec = getPCA(corr_matrix)

In [213]:
# Calculate parameters for denoising
T, N = returns.shape
q = T / N
eMax, var = findMaxEval(np.diag(eVal), q, bWidth=0.01)
nFacts = eVal.shape[0] - np.diag(eVal)[::-1].searchsorted(eMax)

In [214]:
# Denoise and detone
corr_denoised = denoisedCorr(eVal, eVec, nFacts)
cov_denoised = corr_denoised * np.outer(np.sqrt(np.diag(cov_matrix)), np.sqrt(np.diag(cov_matrix)))

In [215]:
# Re-optimize Portfolio with Denoised Covariance
def optimize_portfolio(exp_returns, cov_matrix):
    n = len(exp_returns)
    w = cp.Variable(n)
    risk = cp.quad_form(w, cov_matrix)
    objective = cp.Maximize(exp_returns.values.T @ w - 0.5 * risk)
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return w.value.round(4)

exp_returns = returns.mean() * 252
improved_weights = optimize_portfolio(exp_returns, cov_denoised)

print("Original Weights:", best_weights)
print("Denoised Weights:", improved_weights)

Original Weights: [-2.8957 -0.4408  3.1467  2.3461 -2.4048 -0.2035  1.452 ]
Denoised Weights: [-0.      0.      0.0321  0.9679  0.      0.      0.    ]


In [216]:
def calculate_metrics(weights, returns, cov_matrix, risk_free_rate=0.0453):
    portfolio_returns = np.dot(returns, weights)

    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))

    portfolio_vol = np.sqrt(portfolio_variance)
    sharpe = (portfolio_returns - risk_free_rate) / portfolio_vol

    cumulative_returns = pd.Series(1 + portfolio_returns).cumprod()
    total_portfolio_return = cumulative_returns.iloc[-1] - 1

    return total_portfolio_return, portfolio_variance, sharpe

# Original portfolio metrics
Ret_orig, var_orig, sharpe_orig= calculate_metrics(best_weights, exp_returns.values, cov_matrix)

# Denoised portfolio metrics
Ret_denoised, var_denoised, sharpe_denoised= calculate_metrics(improved_weights, exp_returns.values, cov_denoised)

print(f"Original Portfolio: Portfolio Return={Ret_orig:.2%}, Portfolio Variance={var_orig:.2%}, Sharpe={sharpe_orig:.2f}")
print(f"Denoised Portfolio: Portfolio Return={Ret_denoised:.2%}, Portfolio Variance={var_denoised:.2%}, Sharpe={sharpe_denoised:.2f}")

Original Portfolio: Portfolio Return=242.83%, Portfolio Variance=226.02%, Sharpe=1.59
Denoised Portfolio: Portfolio Return=65.26%, Portfolio Variance=28.87%, Sharpe=1.13


In [217]:
def max_drawdown(returns, weights):
  portfolio_returns = np.dot(returns, weights)

  cumulative_returns = pd.Series(1 + portfolio_returns).cumprod()
  total_portfolio_return = cumulative_returns.iloc[-1] - 1

  peak = cumulative_returns.expanding(min_periods=1).max()
  drawdown = (cumulative_returns - peak) / peak
  max_drawdown = drawdown.min()

  return max_drawdown

md_orig = max_drawdown(returns.values, best_weights)
md_denoised = max_drawdown(returns.values, improved_weights)

print(f"Original Portfolio: Max Drawdown={md_orig:.2%}")
print(f"Denoised Portfolio: Max Drawdown={md_denoised:.2%}")


Original Portfolio: Max Drawdown=-86.46%
Denoised Portfolio: Max Drawdown=-59.97%


### Combination of Denosing and Clustering

In [218]:
def denoise_covariance(cov_matrix, returns):
    corr_matrix = cov2corr(cov_matrix)
    eVal, eVec = getPCA(corr_matrix)
    T, N = returns.shape
    q = T / N
    eMax, var = findMaxEval(np.diag(eVal), q, bWidth=0.01)
    nFacts = eVal.shape[0] - np.diag(eVal)[::-1].searchsorted(eMax)
    eVal_denoised = np.diag(eVal).copy()
    eVal_denoised[nFacts:] = eVal_denoised[nFacts:].sum() / (eVal_denoised.shape[0] - nFacts)
    corr_denoised = eVec @ np.diag(eVal_denoised) @ eVec.T
    cov_denoised = corr_denoised * np.outer(np.sqrt(np.diag(cov_matrix)), np.sqrt(np.diag(cov_matrix)))
    return cov_denoised

# Denoise covariance matrix
cov_denoised = denoise_covariance(cov_matrix, returns.values)

In [219]:
# Hierarchical Clustering

def hierarchical_clustering(cov_matrix, n_clusters=3):
    corr_matrix = pd.DataFrame(cov_matrix).corr().values
    distance_matrix = 1 - np.abs(corr_matrix)  # Use absolute correlation as similarity
    model = AgglomerativeClustering(n_clusters=n_clusters, metric='precomputed', linkage='complete')
    clusters = model.fit_predict(distance_matrix)
    return clusters

# Cluster assets into 3 groups
clusters = hierarchical_clustering(cov_denoised, n_clusters=3)
print("Asset Clusters:", list(zip(tickers, clusters)))

Asset Clusters: [('AAPL', 0), ('NVDA', 0), ('TSLA', 1), ('XOM', 0), ('REGN', 2), ('LLY', 0), ('JPM', 1)]


In [220]:
# Optimize Portfolio with Clusters

def risk_parity_within_cluster(cov_cluster):
    n = cov_cluster.shape[0]
    w = cp.Variable(n)
    risk = cp.quad_form(w, cov_cluster)
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(cp.Minimize(risk), constraints)
    prob.solve()
    return w.value.round(4)

# Split covariance matrix by clusters
cluster_indices = {cluster: np.where(clusters == cluster)[0] for cluster in np.unique(clusters)}
cluster_weights = {}

# Risk parity within each cluster
for cluster, indices in cluster_indices.items():
    cov_cluster = cov_denoised[np.ix_(indices, indices)]
    weights_cluster = risk_parity_within_cluster(cov_cluster)
    cluster_weights[cluster] = (indices, weights_cluster)

# Mean-variance optimization between clusters
# Aggregate cluster returns and covariance
cluster_returns = []
cluster_cov = []
for cluster, (indices, weights) in cluster_weights.items():
    cluster_returns.append(np.dot(returns.iloc[:, indices].mean(), weights))
    cov_cluster = cov_denoised[np.ix_(indices, indices)]
    cluster_cov.append(weights.T @ cov_cluster @ weights)

cluster_returns = np.array(cluster_returns)
cluster_cov = np.diag(cluster_cov)

# Optimize weights across clusters
w_clusters = cp.Variable(len(cluster_returns))
objective = cp.Maximize(cluster_returns.T @ w_clusters - 0.5 * cp.quad_form(w_clusters, cluster_cov))
constraints = [cp.sum(w_clusters) == 1, w_clusters >= 0]
prob = cp.Problem(objective, constraints)
prob.solve()
w_clusters = w_clusters.value.round(4)

# Combine cluster weights with intra-cluster weights
final_weights = np.zeros(len(tickers))
for (cluster, (indices, weights)), w_cluster in zip(cluster_weights.items(), w_clusters):
    final_weights[indices] += weights * w_cluster

print("Final Weights (Denoised + Clustered):", final_weights)

Final Weights (Denoised + Clustered): [0.12442581 0.19867419 0.19918288 0.         0.2315     0.
 0.24621712]


In [221]:
# Compare Metrics

def calculate_metrics(weights, returns, cov_matrix, risk_free_rate=0.02):
    portfolio_returns = np.dot(returns, weights)
    vol = np.sqrt(weights.T @ cov_matrix @ weights)
    sharpe = (portfolio_returns.mean() * 252 - risk_free_rate) / vol
    cumulative = pd.Series(1 + portfolio_returns).cumprod()
    peak = cumulative.expanding().max()
    drawdown = (cumulative - peak) / peak
    max_dd = drawdown.min()
    return sharpe, max_dd, vol

# Original portfolio (GWP2)
sharpe_orig, max_dd_orig, vol_orig = calculate_metrics(best_weights, returns.values, cov_matrix)

# Denoised-only portfolio
sharpe_denoised, max_dd_denoised, vol_denoised = calculate_metrics(improved_weights, returns.values, cov_denoised)

# Clustered-only portfolio (replace with your clustering-only weights)
sharpe_clustered, max_dd_clustered, vol_clustered = calculate_metrics(final_weights, returns.values, cov_matrix)

# Combined method (denoised + clustered)
sharpe_combined, max_dd_combined, vol_combined = calculate_metrics(final_weights, returns.values, cov_denoised)

print("Sharpe Ratios:")
print(f"Original: {sharpe_orig:.2f}, Denoised: {sharpe_denoised:.2f}, Clustered: {sharpe_clustered:.2f}, Combined: {sharpe_combined:.2f}")
print("Volatiltity:")
print(f"Original: {vol_orig:.2%}, Denoised: {vol_denoised:.2%}, Clustered: {vol_clustered:.2%}, Combined: {vol_combined:.2%}")
print("Max Drawdowns:")
print(f"Original: {max_dd_orig:.2%}, Denoised: {max_dd_denoised:.2%}, Clustered: {max_dd_clustered:.2%}, Combined: {max_dd_combined:.2%}")

Sharpe Ratios:
Original: 1.60, Denoised: 1.11, Clustered: 1.17, Combined: 1.24
Volatiltity:
Original: 150.34%, Denoised: 57.02%, Clustered: 16.60%, Combined: 15.66%
Max Drawdowns:
Original: -86.46%, Denoised: -59.97%, Clustered: -15.17%, Combined: -15.17%


In [222]:
def portfolio_retruns(weights, returns):
    return np.dot(returns, weights)

# Original portfolio (GWP2)
return_og = portfolio_retruns(best_weights, exp_returns.values)

# Denoised-only portfolio
return_deno = portfolio_retruns(improved_weights, exp_returns.values)

# Clustered-only portfolio (replace with your clustering-only weights)
return_cluster = portfolio_retruns(final_weights, exp_returns.values)

# Combined method (denoised + clustered)
return_comb = portfolio_retruns(final_weights, exp_returns.values)

print("Portfolio Returns:")
print(f"Original: {return_og:.2%}, Denoised: {return_deno:.2%}, Clustered: {return_cluster:.2%}, Combined: {return_comb:.2%}")

Portfolio Returns:
Original: 242.83%, Denoised: 65.26%, Clustered: 21.41%, Combined: 21.41%


## Clustering

In [223]:
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from sklearn.model_selection import KFold
from itertools import combinations


#### Downloading data using the GWP2 constraints

In [224]:


# Fetch data
tickers = ['AAPL', 'NVDA', 'TSLA', 'XOM', 'REGN', 'LLY', 'JPM']
data = yf.download(tickers, start='2022-01-01', end='2024-12-31', auto_adjust=False)['Adj Close']
returns = data.pct_change().dropna()

# Define Performance Metrics
def sharpe_ratio(returns, risk_free_rate=0.02):
    # print(returns.mean())
    # print(returns.std())
    return (returns.mean() - risk_free_rate) / returns.std()

def max_drawdown(returns):
    cumulative = (1 + returns).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    return drawdown.min()

def expected_shortfall(returns, alpha=0.05):
    return returns[returns <= returns.quantile(alpha)].mean()

# Mean-Variance Optimization Function
def mean_variance_optimization(returns):
    n_assets = returns.shape[1]
    mu = returns.mean()*252
    Sigma = returns.cov()*252
    # print(mu,Sigma)
    w = cp.Variable(n_assets)
    risk_aversion = 1.0
    objective = cp.Maximize(mu.values @ w - risk_aversion * cp.quad_form(w, Sigma))
    constraints = [cp.sum(w) == 1, w >= 0]  # No short-selling constraint
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return w.value




[*********************100%***********************]  7 of 7 completed


#### Walk-Forward Backtesting with MVO

In [225]:
# Walk-Forward Backtesting with MVO
def walk_forward_backtest(returns, window=50):
    wf_results = []
    best_weights_list = []
    for start in range(0, len(returns) - window, window):
        train = returns.iloc[start:start+window]
        test = returns.iloc[start+window:start+2*window]

        if test.empty:
            continue  # Skip if test set is empty

        # Optimize portfolio weights using MVO
        optimal_weights = mean_variance_optimization(train)

        if optimal_weights.shape[0] != returns.shape[1]:
            continue  # Skip if weight dimensions don't match assets

        portfolio_returns = test.dot(optimal_weights)
        best_weights_list.append(optimal_weights)

        wf_results.append({
            "Sharpe": sharpe_ratio(portfolio_returns),
            "Max_Drawdown": max_drawdown(portfolio_returns),
            "Expected_Shortfall": expected_shortfall(portfolio_returns),
            'Volatility': portfolio_returns.std(),
        })
    return pd.DataFrame(wf_results), best_weights_list



In [226]:
# Function to Compute Average Weights
def compute_average_weights(weights_list):
    return np.mean(weights_list, axis=0) if weights_list else np.array([])

# Function to Compute Overall Sharpe Ratio
def overall_sharpe_ratio(returns):
    return sharpe_ratio(returns)


In [227]:

wf_results_df, best_weights = walk_forward_backtest(returns)
wf_results_df, best_weights = walk_forward_backtest(returns)
avg_wf_weights = compute_average_weights(best_weights)
# Display Results
print("Walk-Forward Backtest Results with MVO:")
print(wf_results_df)
print("\nOptimal Weights from Backtesting:")
for i, weights in enumerate(best_weights):
    print(f"Period {i+1}: {weights.round(4)}")

Walk-Forward Backtest Results with MVO:
      Sharpe  Max_Drawdown  Expected_Shortfall  Volatility
0  -0.695370     -0.078953           -0.048999    0.021560
1  -0.792149     -0.205087           -0.048544    0.026446
2  -0.903407     -0.292924           -0.059215    0.026286
3  -1.437405     -0.068949           -0.027284    0.013081
4  -0.480515     -0.076015           -0.037105    0.027019
5  -0.261807     -0.061648           -0.028012    0.039302
6  -0.675449     -0.072498           -0.047417    0.024368
7  -0.646608     -0.128749           -0.046904    0.031096
8  -1.068911     -0.077312           -0.040425    0.019270
9  -0.726460     -0.034920           -0.019935    0.017288
10 -0.538704     -0.197912           -0.059204    0.031295
11 -0.588878     -0.214749           -0.057438    0.031838
12 -0.806355     -0.114632           -0.047065    0.022834
13 -0.866637     -0.134075           -0.040523    0.022570
14       NaN      0.000000           -0.033012         NaN

Optimal Weights

#### K-Fold Cross-Validation for Portfolio Optimization

In [228]:
# K-Fold Cross-Validation for Portfolio Optimization
def kfold_backtest(returns, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=False)
    kf_results = []
    best_weights_list = []

    for train_idx, test_idx in kf.split(returns):
        train, test = returns.iloc[train_idx], returns.iloc[test_idx]

        # Optimize portfolio weights using MVO
        optimal_weights = mean_variance_optimization(train)

        if optimal_weights.shape[0] != returns.shape[1]:
            continue  # Skip if weight dimensions don't match assets

        portfolio_returns = test.dot(optimal_weights)
        best_weights_list.append(optimal_weights)


        kf_results.append({
            "Sharpe": sharpe_ratio(portfolio_returns),
            "Max_Drawdown": max_drawdown(portfolio_returns),
            "Expected_Shortfall": expected_shortfall(portfolio_returns),
            'Volatility': portfolio_returns.std(),
        })

    return pd.DataFrame(kf_results), best_weights_list



In [229]:
kf_results_df, kf_best_weights = kfold_backtest(returns)
kf_results_df, kf_best_weights = kfold_backtest(returns)
avg_kf_weights = compute_average_weights(kf_best_weights)

print("\nK-Fold Cross-Validation Backtest Results with MVO:")
print(kf_results_df)
print("\nOptimal Weights from K-Fold Backtesting:")
for i, weights in enumerate(kf_best_weights):
    print(f"Fold {i+1}: {weights.round(4)}")



K-Fold Cross-Validation Backtest Results with MVO:
     Sharpe  Max_Drawdown  Expected_Shortfall  Volatility
0 -0.545269     -0.503976           -0.074029    0.041303
1 -0.892037     -0.186644           -0.040360    0.020439
2 -0.903668     -0.098551           -0.028325    0.018889
3 -1.223696     -0.072417           -0.026291    0.014156
4 -0.921900     -0.211725           -0.042803    0.020728

Optimal Weights from K-Fold Backtesting:
Fold 1: [-0. -0. -0.  1. -0. -0. -0.]
Fold 2: [0.     0.     0.5447 0.4553 0.     0.     0.    ]
Fold 3: [ 0.      0.      0.0717  0.6163 -0.      0.      0.312 ]
Fold 4: [ 0.      0.      0.6193  0.1832  0.     -0.      0.1975]
Fold 5: [-0.     -0.      0.5884  0.4116  0.      0.     -0.    ]


#### Combinatorial Purged Cross-Validation (CPCV) for Portfolio Optimization


In [230]:
# Combinatorial Purged Cross-Validation (CPCV) for Portfolio Optimization
def cpcv_backtest(returns, n_splits=5, purge_size=20):
    indices = np.arange(len(returns))
    folds = list(KFold(n_splits=n_splits, shuffle=False).split(indices))
    cpcv_results = []
    best_weights_list = []

    for train_idx, test_idx in folds:
        purged_train_idx = [i for i in train_idx if i not in test_idx[:purge_size]]  # Purge overlapping data
        train, test = returns.iloc[purged_train_idx], returns.iloc[test_idx]

        if test.empty:
            continue

        # Optimize portfolio weights using MVO
        optimal_weights = mean_variance_optimization(train)

        if optimal_weights.shape[0] != returns.shape[1]:
            continue

        portfolio_returns = test.dot(optimal_weights)
        best_weights_list.append(optimal_weights)


        cpcv_results.append({
            "Sharpe": sharpe_ratio(portfolio_returns),
            "Max_Drawdown": max_drawdown(portfolio_returns),
            "Expected_Shortfall": expected_shortfall(portfolio_returns),
            'Volatility': portfolio_returns.std(),
        })

    return pd.DataFrame(cpcv_results), best_weights_list



In [231]:
cpcv_results_df, cpcv_best_weights = cpcv_backtest(returns)
cpcv_results_df, cpcv_best_weights = cpcv_backtest(returns)
avg_cpcv_weights = compute_average_weights(cpcv_best_weights)
print("\nCPCV Backtest Results with MVO:")
print(cpcv_results_df)
print("\nOptimal Weights from CPCV Backtesting:")
for i, weights in enumerate(cpcv_best_weights):
    print(f"Fold {i+1}: {weights.round(4)}")



CPCV Backtest Results with MVO:
     Sharpe  Max_Drawdown  Expected_Shortfall  Volatility
0 -0.545269     -0.503976           -0.074029    0.041303
1 -0.892037     -0.186644           -0.040360    0.020439
2 -0.903668     -0.098551           -0.028325    0.018889
3 -1.223696     -0.072417           -0.026291    0.014156
4 -0.921900     -0.211725           -0.042803    0.020728

Optimal Weights from CPCV Backtesting:
Fold 1: [-0. -0. -0.  1. -0. -0. -0.]
Fold 2: [0.     0.     0.5447 0.4553 0.     0.     0.    ]
Fold 3: [ 0.      0.      0.0717  0.6163 -0.      0.      0.312 ]
Fold 4: [ 0.      0.      0.6193  0.1832  0.     -0.      0.1975]
Fold 5: [-0.     -0.      0.5884  0.4116  0.      0.     -0.    ]


#### Overall result

In [232]:
def calculate_metrics(weights, returns_series, cov_matrix, risk_free_rate=0.0453):
    # Portfolio returns (time series)
    portfolio_returns = np.dot(returns_series, weights)

    # Annualized return (geometric mean)
    annualized_return = (1 + portfolio_returns).prod()**(252/len(returns_series)) - 1

    # Annualized volatility
    annualized_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Sharpe ratio (annualized)
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_vol

    # Maximum drawdown
    cumulative_returns = pd.Series((1 + portfolio_returns).cumprod())
    peak = cumulative_returns.expanding().max()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()

    return annualized_return, annualized_vol, sharpe_ratio, max_drawdown


In [233]:
# Walk-Forward

print("Walk-Forward Backtest Results with MVO:")
print(wf_results_df)
print(f"Average sharpe_ratio {wf_results_df.Sharpe.sum(axis=0).round(4)/len(wf_results_df.Sharpe.index)}")
print(f"Average max_drawdown {wf_results_df['Max_Drawdown'].sum(axis=0).round(4)/len(wf_results_df['Max_Drawdown'].index)}")
print(f"Average expected_shortfall {wf_results_df['Expected_Shortfall'].sum(axis=0).round(4)/len(wf_results_df['Expected_Shortfall'].index)}")
print(f"Average expected_shortfall {wf_results_df['Volatility'].sum(axis=0).round(4)/len(wf_results_df['Volatility'].index)}")
print("\nAverage Weights from Walk-Forward Backtesting:")
print(avg_wf_weights.round(4))

# Compute Overall Sharpe Ratio
all_returns = returns*avg_wf_weights

cov_matrix = all_returns.dropna().cov()*252
annualized_return, annualized_vol, sharpe_ratio, max_drawdown = calculate_metrics(avg_wf_weights, returns.values, cov_matrix)
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_vol:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")

Walk-Forward Backtest Results with MVO:
      Sharpe  Max_Drawdown  Expected_Shortfall  Volatility
0  -0.695370     -0.078953           -0.048999    0.021560
1  -0.792149     -0.205087           -0.048544    0.026446
2  -0.903407     -0.292924           -0.059215    0.026286
3  -1.437405     -0.068949           -0.027284    0.013081
4  -0.480515     -0.076015           -0.037105    0.027019
5  -0.261807     -0.061648           -0.028012    0.039302
6  -0.675449     -0.072498           -0.047417    0.024368
7  -0.646608     -0.128749           -0.046904    0.031096
8  -1.068911     -0.077312           -0.040425    0.019270
9  -0.726460     -0.034920           -0.019935    0.017288
10 -0.538704     -0.197912           -0.059204    0.031295
11 -0.588878     -0.214749           -0.057438    0.031838
12 -0.806355     -0.114632           -0.047065    0.022834
13 -0.866637     -0.134075           -0.040523    0.022570
14       NaN      0.000000           -0.033012         NaN
Average sharpe_r

In [234]:


# K-Fold Cross-Validation

print("\nK-Fold Cross-Validation Backtest Results with MVO:")
print(kf_results_df)
print(f"Average sharpe_ratio {kf_results_df.Sharpe.sum(axis=0).round(4)/len(kf_results_df.Sharpe.index)}")
print(f"Average max_drawdown {kf_results_df['Max_Drawdown'].sum(axis=0).round(4)/len(kf_results_df['Max_Drawdown'].index)}")
print(f"Average expected_shortfall {kf_results_df['Expected_Shortfall'].sum(axis=0).round(4)/len(kf_results_df['Expected_Shortfall'].index)}")
print("\nAverage Weights from K-Fold Backtesting:")
print(avg_kf_weights.round(4))


# Compute Overall Sharpe Ratio
all_returns = returns*avg_kf_weights

cov_matrix = all_returns.dropna().cov()*252
annualized_return, annualized_vol, sharpe_ratio, max_drawdown = calculate_metrics(avg_kf_weights, returns.values, cov_matrix)
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_vol:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")


K-Fold Cross-Validation Backtest Results with MVO:
     Sharpe  Max_Drawdown  Expected_Shortfall  Volatility
0 -0.545269     -0.503976           -0.074029    0.041303
1 -0.892037     -0.186644           -0.040360    0.020439
2 -0.903668     -0.098551           -0.028325    0.018889
3 -1.223696     -0.072417           -0.026291    0.014156
4 -0.921900     -0.211725           -0.042803    0.020728
Average sharpe_ratio -0.89732
Average max_drawdown -0.21466
Average expected_shortfall -0.042359999999999995

Average Weights from K-Fold Backtesting:
[ 0.      0.      0.3648  0.5333  0.     -0.      0.1019]
Annualized Return: 59.50%
Annualized Volatility: 17.05%
Sharpe Ratio: 3.22
Maximum Drawdown: -32.77%


In [235]:

# CPCV

print("\nCPCV Backtest Results with MVO:")
print(cpcv_results_df)
print(f"Average sharpe_ratio {cpcv_results_df.Sharpe.sum(axis=0).round(4)/len(cpcv_results_df.Sharpe.index)}")
print(f"Average max_drawdown {cpcv_results_df['Max_Drawdown'].sum(axis=0).round(4)/len(cpcv_results_df['Max_Drawdown'].index)}")
print(f"Average expected_shortfall {cpcv_results_df['Expected_Shortfall'].sum(axis=0).round(4)/len(cpcv_results_df['Expected_Shortfall'].index)}")
print("\nAverage Weights from CPCV Backtesting:")
print(avg_cpcv_weights.round(4))

# Compute Overall Sharpe Ratio
all_returns = returns*avg_cpcv_weights



cov_matrix = all_returns.dropna().cov()*252
annualized_return, annualized_vol, sharpe_ratio, max_drawdown = calculate_metrics(avg_cpcv_weights, returns.values, cov_matrix)
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_vol:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")


CPCV Backtest Results with MVO:
     Sharpe  Max_Drawdown  Expected_Shortfall  Volatility
0 -0.545269     -0.503976           -0.074029    0.041303
1 -0.892037     -0.186644           -0.040360    0.020439
2 -0.903668     -0.098551           -0.028325    0.018889
3 -1.223696     -0.072417           -0.026291    0.014156
4 -0.921900     -0.211725           -0.042803    0.020728
Average sharpe_ratio -0.89732
Average max_drawdown -0.21466
Average expected_shortfall -0.042359999999999995

Average Weights from CPCV Backtesting:
[ 0.      0.      0.3648  0.5333  0.     -0.      0.1019]
Annualized Return: 59.50%
Annualized Volatility: 17.05%
Sharpe Ratio: 3.22
Maximum Drawdown: -32.77%


In [236]:


# GWP2 best weights
best_weights = np.array([-2.8957, -0.4408, 3.1467, 2.3461, -2.4048, -0.2035, 1.4520])

print("\nAverage Weights from GWP2 Backtesting:")
print(best_weights.round(4))







Average Weights from GWP2 Backtesting:
[-2.8957 -0.4408  3.1467  2.3461 -2.4048 -0.2035  1.452 ]


## Backtesting

In [237]:
import yfinance as yf
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [238]:
assets = ["AAPL", "NVDA", "TSLA", "XOM", "REGN", "LLY", "JPM"]

In [239]:
start = datetime.datetime(2022, 1, 1)
end = datetime.datetime(2025, 1, 1)
data = yf.download(assets, start=start, end=end)['Close']

[*********************100%***********************]  7 of 7 completed


In [240]:
data.head()

Ticker            AAPL         JPM         LLY       NVDA        REGN  \
Date                                                                    
2022-01-03  178.879929  147.901260  263.272980  30.070988  626.295166   
2022-01-04  176.609634  153.508133  258.506104  29.241367  616.028381   
2022-01-05  171.911819  150.701691  252.537842  27.558167  594.356201   
2022-01-06  169.042068  152.302734  251.258896  28.131212  597.671936   
2022-01-07  169.209152  153.811752  251.423599  27.201757  602.955139   

Ticker            TSLA        XOM  
Date                               
2022-01-03  399.926666  56.641762  
2022-01-04  383.196655  58.772285  
2022-01-05  362.706665  59.503269  
2022-01-06  354.899994  60.902821  
2022-01-07  342.320007  61.402027

Training the model

In [241]:


def calculate_portfolio_metrics(weights, returns, cov_matrix, risk_free_rate=0.0453):
    portfolio_returns = np.dot(returns, weights)
    annualized_return = (1 + portfolio_returns).prod()**(252/len(returns)) - 1
    annualized_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_vol

    cumulative_returns = pd.Series((1 + portfolio_returns).cumprod())
    peak = cumulative_returns.expanding().max()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()

    return annualized_return, annualized_vol, sharpe_ratio, max_drawdown


annualized_return, annualized_vol, sharpe_ratio, max_drawdown = calculate_portfolio_metrics(final_weights, returns.values, cov_denoised)

print(f"Optimal Portfolio Metrics (using final_weights):")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_vol:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")


Optimal Portfolio Metrics (using final_weights):
Annualized Return: 22.18%
Annualized Volatility: 15.66%
Sharpe Ratio: 1.13
Maximum Drawdown: -15.17%


Testing the model

In [242]:

import yfinance as yf
import datetime

# Define the tickers and the date range
tickers = ['AAPL', 'NVDA', 'TSLA', 'XOM', 'REGN', 'LLY', 'JPM']
start_date = datetime.date(2025, 2, 19)
end_date = datetime.date(2025, 3, 6)

# Download the data
data1 = yf.download(tickers, start=start_date, end=end_date)['Close']

# Calculate daily returns
returns1 = data1.pct_change().dropna()


annualized_return, annualized_vol, sharpe_ratio, max_drawdown = calculate_portfolio_metrics(final_weights, returns1.values, cov_denoised)

print(f"Optimal Portfolio Metrics (using final_weights and new data):")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_vol:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")


[*********************100%***********************]  7 of 7 completed

Optimal Portfolio Metrics (using final_weights and new data):
Annualized Return: -40.99%
Annualized Volatility: 15.66%
Sharpe Ratio: -2.91
Maximum Drawdown: -3.19%


In [243]:
annualized_return, annualized_vol, sharpe_ratio, max_drawdown = calculate_portfolio_metrics(best_weights, returns1.values, sample_cov)
print(f"Optimal Portfolio Metrics (using original_weights, original covariance and new data):")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_vol:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")

Optimal Portfolio Metrics (using original_weights, original covariance and new data):
Annualized Return: -91.13%
Annualized Volatility: 150.34%
Sharpe Ratio: -0.64
Maximum Drawdown: -20.73%


In [244]:
annualized_return, annualized_vol, sharpe_ratio, max_drawdown = calculate_portfolio_metrics(final_weights, returns1.values, sample_cov)
print(f"Optimal Portfolio Metrics (using Clustred_weights, original covariance and new data):")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_vol:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")

Optimal Portfolio Metrics (using Clustred_weights, original covariance and new data):
Annualized Return: -40.99%
Annualized Volatility: 16.60%
Sharpe Ratio: -2.74
Maximum Drawdown: -3.19%


In [245]:
annualized_return, annualized_vol, sharpe_ratio, max_drawdown = calculate_portfolio_metrics(best_weights, returns1.values, cov_denoised)
print(f"Optimal Portfolio Metrics (using original_weights, denoised covariance and new data):")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_vol:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")

Optimal Portfolio Metrics (using original_weights, denoised covariance and new data):
Annualized Return: -91.13%
Annualized Volatility: 168.79%
Sharpe Ratio: -0.57
Maximum Drawdown: -20.73%
